<h1>Target variabele onderzoek.</h1>

Om nuttige voorspellingen te doen, moet eerst bepaald worden wat voorspeld moet worden. Dit document in bedoelt om later in het hoofd-document in integreren, maar is voor nu stand-alone, en dekt de Business-Understanding, Data-Understanding en (gedeeltelijk) Data-Preparation voor het target variabele af.

<h2>Business Understanding: target variabele.</h2>

Door de interviews met onze business-expert hebben begrepen dat het voorspellen van momemt van 'functieherstel' toegevoegde waarde voor ProRail zou hebben. Dit is het moment dat een traject weer klaar is om treinverkeer toe te staan. Via ProRail hebben wij dit diagram aangeleverd gekregen, wat inzicht geeft in het bedrijfsprocess:

![title](images/badkuip.png)
De X-as beslaat tijd, en de Y-as hoeveelheid treinverkeer (beiden non-proportioneel). Dit model zal naar gerefereerd worden als 'badkuip model'.

Functieherstel staat niet expliciet benoemd in dit model, maar vind plaats in de buurt van het "Storing-Eind" moment, en wanneer de lijn weer omhoog begint te bewegen op de Y-as.

ProRail baat bij een goede voorspelling wat betreft moment van functieherstel, zodat ze kunnen zorgen dat het opstarten van de dienstregeling zo goed mogelijk hierop kan aansluiten.

Uit de interviews blijkt dat de gegevens die waarschijnlijk tot een goede prognose leiden waarschijnlijk pas beschikbaar zijn wanneer een aannemer ter plekke is, in het badkuipmodel benoemd als "oplosteam aanwezig". Als het goed is zijn zowel het tijdstip van aankomst van de monteur, als het tijdstip van functieherstel aanwezig in de dataset, hoewel dit later in 'Data Understanding' bevestigd moet worden.

De verstreken tijd tussen de melding van een incident, en het moment dat een oplosteam aanwezig is is sterk variabel. Hoewel dit in de toekomst wellicht ook te voorspellen zou zijn met een model, beperken wij in onze huidige aanpak tot het voorspellen van 'aankomst aannemer' tot 'functieherstel', omdat dat volgens onze huidige business-kennis het best voorspelbaar is aan de hand van de attributen van een specifieke storing.

<h2>Data Understanding: target variabele.</h2>

Om de mogelijkheid van het implementeren van het targetvariabele van (tijdstip functieherstel) - (tijdstip aankomst aannemer) te onderzoeken, moet de data geïnspecteerd om te bepalen of dit ook echt realistisch is. Hiervoor onderzoeken we de eerder benoemde hypothese voor targetvariabele, en eventuele variabelen die hier concurrentie voor zouden kunnen zijn, of meer inzicht zouden kunnen geven.

We beginnen met het importeren van een aantal libraries, en de dataset:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
df = pd.read_csv("sap_storing_data_hu_project.csv")
df

Iets interessants om te onderzoeken is de relatie tussen de tijdstippen van functieherstel, storing eind, en tijdstip van melding. We maken voor de onderzoek doeleinden hier aparte variabelen van voor gemak, en kijken naar de datatypes.

In [ ]:
fh_tijdstip = df["stm_fh_ddt"]
stor_eind = df["stm_sap_storeind_ddt"]
stm_sap_meld_ddt = df["stm_sap_meld_ddt"]

In [ ]:
print(f"""{fh_tijdstip.dtype}
{stor_eind.dtype}
{stm_sap_meld_ddt.dtype}""")

Al deze kolommen bestaan uit Strings. Deze moeten omgezet worden naar DateTime voor effectief rekenwerk. Hierna slaan we ze op in een apart DataFrame voor onderzoek. Uit dit dataframe verwijderen we alle rijen waar minimaal 1 NaN waarde instaat, omdat dit dataframe bedoelt is om de werking tussen de verschillende kolommen te vergelijken.

Ook definieren we een TimeDelta object voor 2 minuten, omdat het exact vergelijken van tijdstippen voor onze doelen vaak niet nuttig is, en als dingen binnen 2 minuten van elkaar vallen, dit in veel gevallen meer informatie kan geven.

In [ ]:
fh_tijdstip = pd.to_datetime(fh_tijdstip, errors="coerce")
stor_eind = pd.to_datetime(stor_eind, errors="coerce")
stm_sap_meld_ddt = pd.to_datetime(stm_sap_meld_ddt, errors="coerce")

twominutes = pd.to_timedelta(2, unit="m")

In [ ]:
#df_de_1 = DataFrame DataExploration 1
df_de_1 = pd.DataFrame({"fh_ddt": fh_tijdstip,
                       "se_ddt": stor_eind,
                       "meld_ddt": stm_sap_meld_ddt}).dropna()

df_de_1.shape  # 1/3rd is dropped from nas

Een eerste vraag om te onderzoek is naar het verschil tussen "Storing eind" en "Functieherstel". Uit ons business-onderzoek begrijpen we dat deze waarschijnlijk de volgende betekenissen hebben:

Functieherstel: Het moment dat er theoretisch weer treinverkeer over het traject kan.
Storing eind: Wanneer het probleem opgelost is, en de herstelwerkzaamheden voltooid zijn.

Omdat het probleem vaak opgelost is, wanneer er weer treinen over het traject kunnen, zullen deze 2 momenten vaak tegelijk zijn. Dit kunnen we ook bevestigen:

In [ ]:
def percentage(numerator: float, denominator: float) -> float:
    """Calculate a percentage from a fraction.

    Args:
        numerator: fraction numerator.
        denominator: fraction denominator.

    Returns:
        Percentage between 0 and 100, as float."""
    return (numerator / denominator) * 100


n_fh_is_se = (np.abs(df_de_1["fh_ddt"] - df_de_1["se_ddt"]) < twominutes).sum()
f"Storing Eind is in {percentage(n_fh_is_se, df_de_1.shape[0])}% van de gevallen binnen 2 minuten van Functie Herstel."

In veel andere gevallen zou men verwachten dat storing eind na functieherstel is, wanneer de aannemer het spoor weer operationeel krijgt, maar daarna wel nog werk moet afmaken.

In [ ]:
n_fh_voor_se = ((df_de_1["se_ddt"] - twominutes) > df_de_1["fh_ddt"]).sum()
f"Storing eind is in {percentage(n_fh_voor_se, df_de_1.shape[0])}% van de gevallen meer dan 2 minuten na functieherstel."

Dit correspondeerd met de verwachting van Business Understanding.

Eerder hebben wij op basis van businesskennis een belovend targetvariabele vastgesteld, namelijk het verschil in tijd tussen het arriveren van de aannemer, en functieherstel. Om dit daadwerkelijk te kunnen gebruiken moet de daadwerkelijke data die hieraan correspondeerd geïnspecteerd worden.

In de data dictionairy staan specifiek een kolom voor datum en tijd (apart) benoemd voor het arriveren van de aannemer, namelijk "stm_aanntpl_dd" voor datum en "stm_aanntpl_tijd" voor tijd. In de dataset zit ook een kolom "stm_aanntpl_ddt", hoewel deze in de dictionairy als n.v.t. bestempeld is. Toch lijkt het nuttig het verschil tussen deze kolommen even te inspecteren, om te kijken welke van deze het beste is.

Ten eerste is het handig een beeld te hebben hoeveel NaN waardes er in deze kolommen zitten.

In [ ]:
n_nan_aanntpl_dd = df["stm_aanntpl_dd"].isna().sum()
f"De datumkolom bestaat voor {percentage(n_nan_aanntpl_dd, df.shape[0])}% uit NaN waardes."

In [ ]:
n_nan_aanntpl_tijd = df["stm_aanntpl_tijd"].isna().sum()
f"De tijdkolom bestaat voor {percentage(n_nan_aanntpl_tijd, df.shape[0])}% uit NaN waardes."

In [ ]:
n_nan_aanntpl_ddt = df["stm_aanntpl_ddt"].isna().sum()
f"De gecombineerde datum en tijdkolom bestaat voor {percentage(n_nan_aanntpl_dd, df.shape[0])}% uit NaN waardes."

In [ ]:
wonky_format = df["stm_aanntpl_dd"] + " " + df["stm_aanntpl_tijd"]

df_aan = pd.DataFrame({"t": pd.to_datetime(df["stm_aanntpl_tijd"], errors="coerce"),
                       "d": pd.to_datetime(df["stm_aanntpl_dd"], errors="coerce"),
                       "dt": pd.to_datetime(df["stm_aanntpl_ddt"], errors="coerce"),
                       "dt_wonky": pd.to_datetime(wonky_format, errors="coerce")}).dropna()

df_aan.shape

In [ ]:
df_aan

In [ ]:
(df_aan["dt"] == df_aan["dt_wonky"]).sum()

We kunnen dus als het goed is de ddt kolom dus gewoon gebruiken...

Dan nog even kijken of er idd genoeg nuttige entries zijn voor functieherstel - arrival monteur

In [ ]:
possible_target = df_de_1["fh_ddt"] - df_aan["dt"]

In [ ]:
possible_target.shape

In [ ]:
possible_target.isna().sum()

In [ ]:
possible_target.dropna(inplace=True)

In [ ]:
target_secs = possible_target.dt.total_seconds()
(target_secs == 0).sum()

In [ ]:
target_secs = target_secs.loc[(target_secs > 1) | (target_secs < -1)]

In [ ]:
(target_secs == 0).sum()

In [ ]:
target_secs

In [ ]:
np.max(target_secs)

In [ ]:
def remove_outliers(s: pd.Series, fence: float = 3) -> pd.Series:
    """Remove the outliers from a Pandas series by removing all values that lie below q1 or above q3 with
    the product of the inter quartile distance and the 'fence'.

    Args:
        s: Series to remove outliers from.
        fence: amount of IQRs value should lie above q3 or below q1 with to get designated an outlier.

    Returns:
        Series without outliers."""
    q1, q3 = s.quantile(.25), s.quantile(.75)
    iqr = q3 - q1
    fence_iqr_prod = fence * iqr
    return s.loc[(s > q1 - fence_iqr_prod) & (s < q3 + fence_iqr_prod)]

target_secs = remove_outliers(target_secs)

In [ ]:
target_secs.hist(bins=50)

In [ ]:
plt.boxplot(target_secs)

In [ ]:
target_secs.shape

Als dit inderdaad een goed targetvariabele is, blijven er 560K van de 890K rows over.

TEMP: FF snel de RMSE van de baseline op basis van target mean.

In [ ]:
baseline_predictions = np.full(target_secs.shape[0], target_secs.mean())
baseline_rmse = metrics.mean_squared_error(target_secs, baseline_predictions, squared=False)
baseline_rmse

In [ ]:
np.max(target_secs)